In [2]:
from config import sparqlTerms, mig_ns, sparqlData, dates
from Titles import date_from_title
from SPARQLWrapper import JSON, SPARQLWrapper
import re, os, json, requests

In [ ]:
sparql = SPARQLWrapper(sparqlData)
sparql.setMethod("POST")
sparql.setReturnFormat(JSON)
query = "select ?s ?o ?agent ?col ?comm where {?s <http://www.w3.org/ns/auth/acl#agent> ?agent . ?s <http://www.w3.org/ns/auth/acl#mode> <http://www.w3.org/ns/auth/acl#Read>. ?s <http://www.w3.org/ns/auth/acl#accessTo> ?o . optional {?o <http://terms.library.ualberta.ca/identifiers/hasCollection> ?col} . optional {?o <http://terms.library.ualberta.ca/identifiers/belongsToCommunity> ?comm } . filter(?agent != <http://projecthydra.org/ns/auth/group#public>)}"
sparql.setQuery(query)
r  = sparql.query().convert()['results']['bindings']
for triple in r:
    print (r)
#with open('CCID.tsv', 'w+') as p:
 #   p.write("Resource" + " \t " + "Community" + " \t " + "Colletion" + " \t " + "Agent" + " \t " + "Permission Object" + "\n")
  #  for triple in r:
   #     p.write(triple['o']['value'] + " \t " + triple['comm']['value'] + " \t " + triple['col']['value'] + " \t " + triple['agent']['value'] + " \t " + triple['s']['value'] + "\n")
print ("done")    

In [ ]:
#PRIVATE ITEMS

In [24]:
sparql = SPARQLWrapper(sparqlData)
sparql.setMethod("POST")
sparql.setReturnFormat(JSON)
query = "prefix xsd: <http://www.w3.org/2001/XMLSchema#> prefix dcterm: <http://purl.org/dc/terms/> select ?s ?o ?agent ?col ?comm where {?s <http://www.w3.org/ns/auth/acl#agent> ?agent . ?s <http://www.w3.org/ns/auth/acl#mode> <http://www.w3.org/ns/auth/acl#Read>. ?s <http://www.w3.org/ns/auth/acl#accessTo> ?o . optional {?o <http://terms.library.ualberta.ca/identifiers/hasCollection> ?col} . optional {?o <http://terms.library.ualberta.ca/identifiers/belongsToCommunity> ?comm} . filter(?agent != <http://projecthydra.org/ns/auth/group#public>)}"
sparql.setQuery(query)
r  = sparql.query().convert()['results']['bindings']
with open('CCID_4.tsv', 'w+') as p:
    p.write("Resource" + " \t " + "Agent" + " \t " + "CommunityID" + " \t "  + "Community Title" + " \t "  + "Collection" + " \t " + "Permission Object" + "\n")
    for triple in r:
        if "comm" in triple.keys() and "col" in triple.keys():
            comm = triple['comm']['value']
            subject = "http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/{}/{}/{}/{}/{}".format(comm[0:2], comm[2:4], comm[4:6], comm[6:8], comm)
            query = "prefix xsd: <http://www.w3.org/2001/XMLSchema#> prefix dcterm: <http://purl.org/dc/terms/> select ?title where {<%s> <http://fedora.info/definitions/v4/repository#hasParent> ?par . optional{<%s> dcterm:title ?title}}" % (subject, subject)
            sparql.setQuery(query)
            re  = sparql.query().convert()['results']['bindings']
            for tri in re:
                if "title" in tri.keys():
                    p.write(triple['o']['value'] + " \t " + triple['agent']['value'] + " \t " + triple['comm']['value'] + " \t " + tri["title"]["value"] + " \t " + triple['col']['value'] + " \t " + triple['s']['value'] + "\n")
                else:
                    p.write(triple['o']['value'] + " \t " + triple['agent']['value'] + " \t " + triple['comm']['value'] + " \t " + "No title for the community was found" + " \t " + triple['col']['value'] + " \t " + triple['s']['value'] + "\n")
        elif "comm" in triple.keys() and "col" not in triple.keys():
            comm = triple['comm']['value']
            subject = "http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/{}/{}/{}/{}/{}".format(comm[0:2], comm[2:4], comm[4:6], comm[6:8], comm)
            query = "prefix xsd: <http://www.w3.org/2001/XMLSchema#> prefix dcterm: <http://purl.org/dc/terms/> select ?title where {<%s> <http://fedora.info/definitions/v4/repository#hasParent> ?par . optional{<%s> dcterm:title ?title}}" % (subject, subject)
            sparql.setQuery(query)
            re  = sparql.query().convert()['results']['bindings']
            for tri in re:
                if "title" in tri.keys():
                    p.write(triple['o']['value'] + " \t " + triple['agent']['value'] + " \t " + triple['comm']['value'] + " \t " + tri["title"]["value"] + " \t " + "No Collection" + " \t " + triple['s']['value'] + "\n")
                else:
                    p.write(triple['o']['value'] + " \t " + triple['agent']['value'] + " \t " + triple['comm']['value'] + " \t " + "No title for the community was found" + " \t " + "No Collection" + " \t " + triple['s']['value'] + "\n")
        elif "comm" not in triple.keys() and "col" not in triple.keys():
            p.write(triple['o']['value'] + " \t " + triple['agent']['value'] + " \t " + "Item has no community" + " \t " + "No title for the community was found" + " \t " + "No Collection" + " \t " + triple['s']['value'] + "\n")
print ("done") 

done


In [34]:
o = []
t = []
with open("1.txt", "r") as one:
    for line in one:
        line = line.replace("\n", "")
        if line not in o:
            o.append(line)
with open("2.txt", "r") as two:
    for lin in two:
        lin = lin.replace("\n", "")
        if lin not in o:
            t.append(lin)
for i in t:
    query = "prefix xsd: <http://www.w3.org/2001/XMLSchema#> prefix dcterm: <http://purl.org/dc/terms/> select ?comm ?col where {<%s> <http://terms.library.library.ca/identifiers/belongsToCommunity> ?comm . <%s> <http://terms.library.library.ca/identifiers/hasCollection> ?col}" % (i, i)
    sparql.setQuery(query)
    re  = sparql.query().convert()['results']['bindings']
    with open("diff.tsv", "a") as out:
        for tri in re:
             out.write(i + "\t" + tri['comm']['value'] + "\t" + tri['col']['value'] + "\n")
print ("done")

done
